In [2]:
import numpy as np
import os
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import glob
import matplotlib.pyplot as plt
import tqdm
import pywt
import scipy.signal
import scipy.ndimage
import ecg
import tqdm
import joblib
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

In [ ]:
guohuan = pd.read_csv(r'D:\\byx\\baseline\\myEcg\\data\\过缓.csv')

In [ ]:
for index,age in zip(guohuan['index'],guohuan['age']):
    print(index,age)

## Test

In [ ]:
TP,FN = 0,0
error = []
for txtname,age in zip(guohuan['index'],guohuan['age']):
    df = pd.read_csv(r'D:\\byx\\baseline\\myEcg\\data\\hf_round2_train\\%s'%txtname,delimiter=' ')
    sampling_rate = 500
    if ecg.raise_error(df['I']):
        print('error',txtname)
        continue
    #peaks = R_peak(df,plot=False)
    _,_,epochs = ecg.segment(df,plot=False)
    #print((np.mean(r['T_Onsets']) - np.mean(r['P_Offsets'])) / 500)
    #print('心率：',60 / len(epochs[0]) * sampling_rate)
    xinlv = 60 / len(epochs[0]) * sampling_rate
    if xinlv > 60:
        #print(txtname)
        #peaks = ecg.R_peak(df,plot=True)
        #error.append(txtname)
        FN += 1
    else:
        TP += 1
print(TP/(TP+FN))

In [ ]:
FP,TN = 0,0
trains = glob.glob(r'D:\\byx\\baseline\\myEcg\\data\\hf_round2_train\\**.txt')
c = 0
for txtname in tqdm.tqdm(trains[:100]):
    txtn = txtname.split('\\')[-1]
    if txtn not in guohuan['index']:
        df = pd.read_csv(txtname,delimiter=' ')
        if ecg.raise_error(df['I']):
            print('error',txtname)
            continue
        _,_,epochs,_ = ecg.segment(df,plot=False)
        xinlv = 60 / len(epochs[0]) * sampling_rate
        if xinlv > 60:
            #print(txtname)
            #peaks = ecg.R_peak(df,plot=True)
            #error.append(txtname)
            TN += 1
        else:
            FP += 1
            peaks = ecg.R_peak(df,plot=True)
            print(xinlv)

In [ ]:
Precision = TP/(TP+FP)
Recall = TP/(TP+FN)
F1 = 2 * (Precision*Recall)/(Precision+Recall)
Accuracy = (TP+TN) / (TP+FP+FN+TN)
print(f'Precision = {Precision}')
print(f'Recall = {Recall}')
print(f'F1 = {F1}')
print(f'Accuracy = {Accuracy}')

# building dataset

In [ ]:
def heart_rating(txtname,sampling_rate=500):
    df = pd.read_csv(r'D:\\byx\\baseline\\myEcg\\data\\hf_round2_train\\%s'%txtname,delimiter=' ')
    if ecg.raise_error(df['I']):
        print('error',txtname)
        return np.nan
    _,_,epochs,_ = ecg.segment(df,plot=False)
    xinlv = 60 / len(epochs[0]) * sampling_rate
    return xinlv

In [ ]:
dataset = []
root = 'D:\\byx\\baseline\\myEcg\\data'
train_csv = os.path.join(root,'hf_round2_label_train.csv')
train = pd.read_csv(train_csv)
train = train.dropna(axis=0,how='any')  

In [ ]:
train['one_label'] = train['one_label'].map(lambda x: eval(x))
train['guohuan'] = train['one_label'].map(lambda x:1 if x[18]==1.0 else 0)
train['heart_ratings'] = train['index'].map(lambda x:heart_rating(x))
train = train.dropna(axis=0,how='any')  

In [ ]:
train['sex2'] = train['sex'].map(lambda x:1 if x=='MALE' else 0)
train_size = int(len(train)*0.5)
train_dataset = train[:train_size]
test_dataset = train[train_size:]
X_train = train_dataset[['age','heart_ratings','sex2']]
y_train = train_dataset['guohuan']
X_test = test_dataset[['age','heart_ratings','sex2']]
y_test = test_dataset['guohuan']

In [ ]:
train.to_csv('train_heartrating.csv')

In [ ]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pre = clf.predict(X_test)
recall = recall_score(y_test,y_pre)
accuracy = accuracy_score(y_test,y_pre)
precision = precision_score(y_test,y_pre)
F1 = f1_score(y_test,y_pre)
print(f'Precision = {precision}')
print(f'*Recall* = {recall}')
print(f'F1 = {F1}')
print(f'Accuracy = {accuracy}')

In [ ]:
train[train['guohuan']==1]

In [ ]:
clf.coef_,clf.intercept_

# 过速

In [ ]:
train['guosu'] = train['one_label'].map(lambda x:1 if x[30]==1.0 else 0)

In [ ]:
train_size = int(len(train)*0.5)
train_dataset = train[:train_size]
test_dataset = train[train_size:]
X_train = train_dataset[['age','heart_ratings']]
y_train = train_dataset['guosu']
X_test = test_dataset[['age','heart_ratings']]
y_test = test_dataset['guosu']

clf = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pre = clf.predict(X_test)
recall = recall_score(y_test,y_pre)
accuracy = accuracy_score(y_test,y_pre)
precision = precision_score(y_test,y_pre)
F1 = f1_score(y_test,y_pre)
print(f'Precision = {precision}')
print(f'*Recall* = {recall}')
print(f'F1 = {F1}')
print(f'Accuracy = {accuracy}')

### 过速 - 东阳数据集

In [5]:
def heart_rating(txtname,sampling_rate=500):
    df = pd.read_csv(r'D:\\byx\\Dongyang\\txt_data\\%s'%txtname,delimiter=' ')
    if ecg.raise_error(df['I']) or ecg.raise_error(df['II']):
        print('error',txtname)
        return np.nan
    try:
        peaks = ecg.R_peak(df,lead='II')
        xinlv = 60/((peaks[-1]-peaks[0])/(len(peaks)-1)/500)
    except Exception as e:
        print('***',txtname)
        return np.nan
    return xinlv

In [3]:
train = pd.read_csv(r'D:\byx\Dongyang\label_add_heartrating_2.csv')
train

,Unnamed: 0,Unnamed: 0.1,0,1,过速,heart_rating,heart_rating_updated
0,0,0,FILE@57f67388ce5ab304d8f0a72acf2339bf.txt,窦性心律 正常心电图范围,0,68.649886,68.464730
1,1,1,FILE@c4505b5ef6c09c1afee93839e2de70ae.txt,窦性心律 正常心电图范围,0,75.000000,74.859638
2,2,2,FILE@7b25b8706476ac7bbaaed23f902dd3b5.txt,窦性心律 正常心电图范围,0,87.463557,87.073007
3,3,3,FILE@12eef1bdec5b766e4466dfd097c28a44.txt,窦性心律 正常心电图范围,0,73.529412,73.349633
4,4,4,FILE@56e7521c27693266d32275536f1ff9e7.txt,1. 窦性心律\n2. 房性早搏,0,71.770335,65.061809
5,5,5,FILE@4fe86725065038f57e4abca7391e5fd4.txt,窦性心律 正常心电图范围,0,66.225166,66.006601
6,6,6,FILE@a5f945fd89f184ba13dc29dec0e81be0.txt,1. 窦性心律\n2. 电轴左偏-39度,0,90.909091,90.869753
7,7,7,FILE@d83051d4ac81bba0d55fc9d2b3e7622e.txt,1. 窦性心律\n2. 完全性左束支传导阻滞\n3. ST-T改变,0,79.787234,65.919578
8,8,8,FILE@e1e98f0230ae026ffa216422e6117aa1.txt,窦性心律 正常心电图范围,0,64.102564,63.938619
9,9,9,FILE@c02b925526cc24e490e453c7a5fc21f9.txt,窦性心律 正常心电图范围,0,76.726343,76.514346


In [ ]:
train = train.dropna(axis=0,how='any')  
train_size = int(len(train)*0.5)
train_dataset = train[:train_size]
test_dataset = train[train_size:]
X_train = train_dataset[['heart_rating']]
y_train = train_dataset['过缓']
X_test = test_dataset[['heart_rating']]
y_test = test_dataset['过缓']

clf = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pre = clf.predict(X_test)
recall = recall_score(y_test,y_pre)
accuracy = accuracy_score(y_test,y_pre)
precision = precision_score(y_test,y_pre)
F1 = f1_score(y_test,y_pre)
print(f'Precision = {precision}')
print(f'*Recall* = {recall}')
print(f'F1 = {F1}')
print(f'Accuracy = {accuracy}')

In [33]:
y_pre = train['heart_rating_updated'].map(lambda x:1 if x <60else 0)
y_test = train['过缓']
recall = recall_score(y_test,y_pre)
accuracy = accuracy_score(y_test,y_pre)
precision = precision_score(y_test,y_pre)
F1 = f1_score(y_test,y_pre)
print(f'Precision = {precision}')
print(f'*Recall* = {recall}')
print(f'F1 = {F1}')
print(f'Accuracy = {accuracy}')

Precision = 0.7511910250499462
*Recall* = 0.9920844327176781
F1 = 0.8549938779079936
Accuracy = 0.9715320822101269


In [34]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test,y_pre).ravel()
tn, fp, fn, tp

(51695, 1619, 39, 4888)